# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
#Polaridad es el sentimiento
# Ver el potencial impacto de la publicidad
# Cuanto cobrar despues de analizar cada noticia
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
#Arbol de decisión
import numpy as np
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=3, random_state=123)
treeclf.fit(X_train, y_train)
y_pred = treeclf.predict(X_test)
print('Accuracy Decision Tree: ', round(metrics.accuracy_score(y_test,y_pred),4))
print('f1_score Decision Tree: ',round(metrics.f1_score(y_test,y_pred),4))

Accuracy Decision Tree:  0.6413
f1_score Decision Tree:  0.6073


In [7]:
#Regresión Logistica
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear',C=1e9,random_state=123)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy Logistic: ', round(metrics.accuracy_score(y_test,y_pred),4))
print('f1_score Logistic: ',round(metrics.f1_score(y_test,y_pred),4))

Accuracy Logistic:  0.622
f1_score Logistic:  0.6049


# Exercise 9.2 Manual

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [8]:
#1. Se establece variable que indica la cantidad de estimaciones que se van a realizar
n_estimators = 300
#2. Se definie variable que corresponde a la cantidad de observaciones
n_samples = X_train.shape[0]
#3. Se construyen 100 arregos de muestras de forma aleatoreas, seleccionando números entre 0 y el tamaño de X_train
#   np.ramdom.choice(a=rango de la muestra,size, tamaño, con remplazo ). Cada arreglo corresponderá a los indices
#   de los registros de X_train que se tomaran en cada modelo que se construya.
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(300)]

In [9]:
from sklearn.tree import DecisionTreeClassifier
#Se generan números aleatorios entre 1 y 10000, para utilizarlos, ya no para construir rango de muestras, sino como semilla del arbol de decision

#Se establece una semilla
np.random.seed(123) 
#Se generan números aleatorios entre 1 y 10000 para utilizar como semilla en random_state, para generar muestras
#aleatorias con semillas distintas.
seeds = np.random.randint(1, 10000, size=100)

#Se crean 300 modelos con muestras aleatorias distintas. 200 modelos de decision tree con max_depth distintos y 
#100 con regresion logistica
#X_train.iloc[samples[i]] con este selecciono, de manera aleatoria, valores de X_train. 
# Samples corresponde al rango de indicies (100 en este caso) para seleccionar los valores o muestras de X_train.

trees={}
for i in range(100):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i]) 
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    trees[i+100] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i])  
    trees[i+100].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    trees[i+200] = LogisticRegression(solver='liblinear',C=1e9,random_state=seeds[i]) 
    trees[i+200].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])



In [10]:
#Se hace una predicción de cada uno de los 300 modelos creados
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)
y_pred_df.head()


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,1,1,1,1,0,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3721,1,0,0,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Se suman los 1 de todos los modelos para tener una idea de cuales se clasifican como predicción positiva
y_pred_df.sum(axis=1)[:X_test.shape[0]]


1483    264
2185    258
2520    240
3721    257
3727     34
4524    119
234      14
4735     48
5839    128
2939    225
3053    247
867     248
276     261
5798     28
3512    269
5789     23
3198    242
5908    272
5718     54
2687     26
250     102
5461    256
3052     30
2711    135
3771     35
5988     30
5645     93
4617     68
368     204
4916     89
       ... 
19       63
4088    243
1395    216
2576     46
2878     86
3147     95
4513     47
1620    117
4522    244
5485    223
2394     17
1691     72
4949    108
2805    123
494     239
5287     24
3316    104
3176    237
2372    236
5947    261
5782     38
1814     56
4704     43
5333     79
946     158
3077    112
5166     57
2227    242
5684     80
1937     38
Length: 1500, dtype: int64

# Exercise 9.3 Manual

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [12]:
#Se seleccionan las predicciones que dieron mas de 50% como positiva.
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

from sklearn import metrics
print('F1 score: ',round(metrics.f1_score(y_pred, y_test),4))
print('Accuracy: ',round(metrics.accuracy_score(y_pred, y_test),4))

F1 score:  0.6286
Accuracy:  0.64


# Exercise 9.2 Sklearn

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [13]:
#Con sklearn
clf={}

from sklearn.ensemble import BaggingClassifier
SCL = ['A','B','C']
for i in SCL:
    if i == 'A':
        clf[0] = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None,random_state=123), n_estimators=100, bootstrap=True,
                            random_state=123, n_jobs=-1, oob_score=True)
        clf[0].fit(X_train, y_train)
    if i == 'B':
        clf[1] = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=2,random_state=123), n_estimators=100, bootstrap=True,
                            random_state=123, n_jobs=-1, oob_score=True)
        clf[1].fit(X_train, y_train)
    if i == 'C':
        clf[2] = BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',C=1e9,random_state=123), n_estimators=100, bootstrap=True,
                            random_state=123, n_jobs=-1, oob_score=True)
        clf[2].fit(X_train, y_train)

# Exercise 9.3 Sklearn

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [14]:
y_pred_skl = pd.Series()

y_pred_skl =  pd.Series(clf[0].predict(X_test))
y_pred_skl.append(pd.Series(clf[1].predict(X_test)))
y_pred_skl.append(pd.Series(clf[2].predict(X_test)))
y_pred_skl=y_pred_skl.tolist()
print('F1 score: ',round(metrics.f1_score(y_pred_skl, y_test),4))
print('Accuracy: ',round(metrics.accuracy_score(y_pred_skl, y_test),4))

F1 score:  0.6334
Accuracy:  0.6327


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score. Modificar threshold diferente a 0.5

In [15]:
# hace una nueva predicción, variando el % que se considera como prediccion positiva
# para la selección de las predicciones
a=0
for i in range(1, 100 , 1):
    y_pred2 = (y_pred_df.sum(axis=1)/300 >= i/100).astype(np.int)
    b=metrics.f1_score(y_pred2, y_test)
    if b>=a:
        a=b
        j=i

print('La probabilidad que mejor maximiza el F1 score es: ',j/100,' con F1 de: ',round(a,4))

La probabilidad que mejor maximiza el F1 score es:  0.19  con F1 de:  0.7081


/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [16]:
samples_oob = []
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample))) 

In [17]:
errors = np.zeros(n_estimators)

for i in range(300):
    y_pred_ = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [18]:
alpha = (1 - errors) / (1 - errors).sum()

In [19]:
weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)

In [20]:
y_pred = (weighted_sum_1 >= 0.5).astype(np.int)

print('F1 score: ',round(metrics.f1_score(y_pred, y_test),4))
print('Accuracy: ',round(metrics.accuracy_score(y_pred, y_test),4))

F1 score:  0.6261
Accuracy:  0.6393


# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [21]:
a=0
for i in range(1, 100 , 1):
    y_pred2 = (weighted_sum_1 >= i/100).astype(np.int)
    b=metrics.f1_score(y_pred2, y_test)
    if b>=a:
        a=b
        j=i
print('La probabilidad que mejor maximiza el F1 score es: ',j/100,' con F1 de: ',round(a,4))

La probabilidad que mejor maximiza el F1 score es:  0.18  con F1 de:  0.7081


/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


# Exercise 9.7 
Staking

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [22]:
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(300)))

for i in range(300):
    X_train_2[i] = trees[i].predict(X_train)

In [23]:
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv=5)
lr.fit(X_train_2, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [24]:
y_pred = lr.predict(y_pred_df)
print('F1 score: ',round(metrics.f1_score(y_pred, y_test),4))
print('Accuracy: ',round(metrics.accuracy_score(y_pred, y_test),4))

F1 score:  0.6511
Accuracy:  0.6507


In [25]:
a=0
for i in range(1, 100 , 1):
    preds = np.where(lr.predict_proba(y_pred_df)[:,1] > i/100, 1, 0)
    b=metrics.f1_score(preds, y_test)
    if b>=a:
        a=b
        j=i
print('La probabilidad que mejor maximiza el F1 score es: ',j/100,' con F1 de: ',round(a,4))


La probabilidad que mejor maximiza el F1 score es:  0.37  con F1 de:  0.7143


/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/sysroot/home/asuspc/anaconda3/lib/python3.7/site-packages/sklearn/metrics/class